<a href="https://colab.research.google.com/github/MiraSharma8511/POC-Model-Training/blob/main/deberta_v3_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/microsoft/deberta-v3-large

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/microsoft/deberta-v3-large)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="microsoft/deberta-v3-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [2]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("microsoft/deberta-v3-large")

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment, running this may bill your account above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, using the **auto** provider setting (automatically selects an available inference provider).

In [9]:
import os
from huggingface_hub import InferenceClient
from google.colab import userdata

api_key_hf = (userdata.get("HF_TOKEN"))

client = InferenceClient(
    provider="auto",
    api_key=api_key_hf,
)

result = client.fill_mask(
    "The answer to the universe is [MASK].",
    model="microsoft/deberta-v3-large",
)

In [10]:
result

[FillMaskOutputElement(score=0.001319719129242003, sequence='The answer to the universe is Bernina .', token=122256, token_str='Bernina', fill_mask_output_token_str=None),
 FillMaskOutputElement(score=0.0007842915947549045, sequence='The answer to the universe is shopper .', token=24041, token_str='shopper', fill_mask_output_token_str=None),
 FillMaskOutputElement(score=0.0007665283628739417, sequence='The answer to the universe is disbursements .', token=76497, token_str='disbursements', fill_mask_output_token_str=None),
 FillMaskOutputElement(score=0.0007129862206056714, sequence='The answer to the universe is Lift .', token=15462, token_str='Lift', fill_mask_output_token_str=None),
 FillMaskOutputElement(score=0.0006472485838457942, sequence='The answer to the universe is chandelier .', token=20147, token_str='chandelier', fill_mask_output_token_str=None)]

In [7]:
!pip install transformers datasets evaluate scikit-learn torch

In [8]:
!pip install transformers datasets scikit-learn torch

In [12]:
import json
import pandas as pd

with open("train_jobs_unique.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df["text"] = df["job"] + " - " + df["task"]
df.head()


,job,task,category,text
0,International Strategist,Processing statement applications,Augment with GenAI,International Strategist - Processing statemen...
1,"Merchandiser, Coach",Drafting occur reports,Human,"Merchandiser, Coach - Drafting occur reports"
2,"Designer, Technician",Managing discover processes,Augment with Automation,"Designer, Technician - Managing discover proce..."
3,Media Assembler,Drafting wife reports,Augment with Automation,Media Assembler - Drafting wife reports
4,Radio Strategist,Generating movie content,Augment with GenAI,Radio Strategist - Generating movie content


In [13]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["category"])


In [14]:
from transformers import AutoTokenizer
from datasets import Dataset

dataset = Dataset.from_pandas(df[["text", "label"]])
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
!pip install transformers datasets scikit-learn torch

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-large", num_labels=len(label_encoder.classes_)
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    eval_strategy="no",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-19-2131803085.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>